In [1]:
import numpy as np
import pandas as pd
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import os
from nltk.parse import stanford

In [2]:
client = ElsClient('506e073160e8aa445febbadf41f11c83') # establish client, whats in the argument is the API key

In [3]:
url_filepath = '/Volumes/My Passport/Davids Stuff/9-26-19-scrape-redo.txt'

In [9]:
#only need to do this if scrape-redo2 doesn't already exist
with open(url_filepath,'r') as file:
    with open('/Volumes/My Passport/Davids Stuff/9-26-19-scrape-redo2.txt','w') as file2:
    
        for entry in file:
            commas = entry.count(',')
            if commas < 2:
                file2.write(entry)
        

In [4]:
df = pd.read_csv('/Volumes/My Passport/Davids Stuff/9-26-19-scrape-redo2.txt') 

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
output_filepath = '/Volumes/My Passport/Davids Stuff/'

In [6]:
df.columns = (['url','year'])
df.head()

,url,year
0,https://www-sciencedirect-com.offcampus.lib.wa...,2014
1,https://www-sciencedirect-com.offcampus.lib.wa...,2014
2,https://www-sciencedirect-com.offcampus.lib.wa...,2014
3,https://www-sciencedirect-com.offcampus.lib.wa...,2014
4,https://www-sciencedirect-com.offcampus.lib.wa...,2014


In [7]:
# This checks to removes the rows that are messed up, usually very rare occurence
# If the two printed values are the same, everything should be consistent between urls and piis gathered
# This is important for properly aligning data downstream
piis = []
for i in range(len(df)):
    try:
        pi = df.iloc[i,0][-17:]
        piis.append(pi)
    except:
        df = df.drop(i,axis=0)

print(len(piis))
print(len(df))
df.head()

2002842
2002842


,url,year
0,https://www-sciencedirect-com.offcampus.lib.wa...,2014
1,https://www-sciencedirect-com.offcampus.lib.wa...,2014
2,https://www-sciencedirect-com.offcampus.lib.wa...,2014
3,https://www-sciencedirect-com.offcampus.lib.wa...,2014
4,https://www-sciencedirect-com.offcampus.lib.wa...,2014


In [8]:
## ScienceDirect (full-text) document example using PII
def get_doc(dtype,identity):
    if dtype == 'pii':
        doc = FullDoc(sd_pii = identity)
    elif dtype == 'doi':
        doc= FullDoc(doi = identity)
       
    if doc.read(client):
            #print ("doc.title: ", doc.title)
            doc.write()   
    else:
        print ("Read document failed.")
    
    return doc
       
        

In [9]:
def authorize(doc):
    #this method takes a doc object and returns a list of authors for the doc
    auths = []
    for auth in doc.data['coredata']['dc:creator']:
        auths.append(auth['$'])
    
    return auths

In [22]:
ab_starters = ['Abstract',' Abstract','abstract',' abstract']

In [10]:
def get_abs(doc):
    abstract = doc.data['coredata']['dc:description']
    
    for starter in ab_starters:
        if abstract.startswith(starter):
            abstract = abstract[len(starter) + 1:]
            break
    
    return abstract

#### Use stuff between these markdown cells if wanting to test different methods for data retrieval from doc object

In [19]:
example_doc = get_doc('pii',piis[41])

In [20]:
authorize(example_doc)

['Yu, Yongzhi',
 'Cheng, Si',
 'Wang, Leying',
 'Zhu, Wenjun',
 'Luo, Linghong',
 'Xu, Xu',
 'Song, Fusheng',
 'Li, Xiaohong',
 'Wang, Jigang']

In [25]:
print(get_abs(example_doc))

A facile and environmentally benign microwave heating approach was developed to prepare yolk-shell porous Fe-doped graphitic carbon nitride (YPFCN) microarchitectures by using low-cost melamine, graphite and iron as raw materials for the first time. The as-synthesized samples were well characterized by various analytical techniques. Results showed that Fe-doping could induce intrinsic electronic and band structure modulation of g-C3N4, resulting in its absorbance edge up to 460 nm, consequently. The derived bandgap of YPFCN was estimated to be 2.71 eV with a decrease of 0.10 eV compared to that of undoped g-C3N4 (2.81 eV). Besides, the YPFCN photocatalyst showed excellent visible-light driven photocatalytic activity toward Rhodamine B (RhB) and Methyl orange (MO) presumably as a result of large surface area, efficient separation of photo-generated carries, shorter carrier lengths, and higher visible-light absorption efficiency. The removal ability of RhB and MO of YPFCN was about 1.2 a

In [26]:
example_doc.data['coredata']

{'eid': '1-s2.0-S2214993718300964',
 'dc:description': 'Abstract A facile and environmentally benign microwave heating approach was developed to prepare yolk-shell porous Fe-doped graphitic carbon nitride (YPFCN) microarchitectures by using low-cost melamine, graphite and iron as raw materials for the first time. The as-synthesized samples were well characterized by various analytical techniques. Results showed that Fe-doping could induce intrinsic electronic and band structure modulation of g-C3N4, resulting in its absorbance edge up to 460\u202fnm, consequently. The derived bandgap of YPFCN was estimated to be 2.71\u202feV with a decrease of 0.10\u202feV compared to that of undoped g-C3N4 (2.81\u202feV). Besides, the YPFCN photocatalyst showed excellent visible-light driven photocatalytic activity toward Rhodamine B (RhB) and Methyl orange (MO) presumably as a result of large surface area, efficient separation of photo-generated carries, shorter carrier lengths, and higher visible-li

### ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

#### Just examples of grabbing data from a single document object

In [114]:
# text=doc.data['originalText']
# auths = authorize(doc)
# issn = doc.data['coredata']['prism:issn']
# journal = doc.data['coredata']['prism:publicationName']
# title = doc.data['coredata']['dc:title']
# abstract = doc.data['coredata']['dc:description']

In [11]:
# one more check before we go
print('length of dataframe is ',len(df))
print('length of piis is      ',len(piis))

length of dataframe is  2002842
length of piis is       2002842


In [14]:
attempts = 0
successes = 0 # a download is considered a success if the full text was grabbed
filenum = 0
success_cutoff = 5000 #how many successful downloads do we want before writing to memory 
ab_starters = ['Abstract',' Abstract','abstract',' abstract'] # possible abstract prefixes for abstracts we'll want to get rid of

os.chdir(output_filepath)                # make sure in correct directory
file = open('corpus_texts_0.txt','w')

for pii in piis:
    attempts += 1
    if attempts % 1000 == 0:
        print('Attempts : ',attempts)
        print('Successes: ',successes)
    
    if attempts < 1000 and attempts % 1 == 0:
        print('Attempts : ',attempts)
        print('Successes: ',successes)
    
    if successes % success_cutoff == 0:
        print('resetting the dynamic lists')
        # reset the dynamic lists every time we hit critical number of successful downloads 
        dyn_auths = []
        dyn_issns = []
        dyn_journals = []
        dyn_titles = []
        dyn_abstracts = []
        dyn_piis = []
        dyn_years = []
        dyn_urls = []
        dyn_successes = []
        
    
    url = df['url'][attempts-1]                              # grab url from previous dataframe
    year = df['year'][attempts-1]                            # grab year from previous dataframe
            
    dyn_piis.append(pii)
    dyn_years.append(year)
    dyn_urls.append(url)
        
    # retrieval of any of these data could potentially fail, so they are wrapped with try/except clauses, and this is extremely redundant for that reason
    try:
        doc = get_doc('pii',pii)                                     # grab document object from elsevier 
        try:
            text = doc.data['originalText']                          # grab original full text                                                        
            file.write(text) 
            file.write('\n')
            dyn_successes.append(1)
            successes += 1
        except:
            file.write('ORIGINAL TEXT EXCEPTION')
            file.write('\n')
            dyn_successes.append(0)
        
        try:
            auths = authorize(doc)                                   # grab list of the authors
            dyn_auths.append(auths)
        except:
            dyn_auths.append(None)
            
        try:
            issn = doc.data['coredata']['prism:issn']                # grab issn of the journal
            dyn_issns.append(issn)
        except:
            dyn_issns.append(None)
        
        try:
            journal = doc.data['coredata']['prism:publicationName']  # grab journal name
            dyn_journals.append(journal)
        except:
            dyn_journals.append(None)
            
        try:
            title = doc.data['coredata']['dc:title']                 # grab title
            dyn_titles.append(title)
        except:
            dyn_titles.append(None)
        
        try:
            abstract = get_abs(doc)        # grab abstract
            dyn_abstracts.append(abstract)
        except:
            dyn_abstracts.append(None)
        
    except:                      # if doc object retrieval fails, append None for everything (except the url and year which were already done above)
        dyn_successes.append(0)
        file.write('DOC OBJECT RETRIEVAL EXCEPTION')
        file.write('\n')
        dyn_auths.append(None)
        dyn_issns.append(None)
        dyn_journals.append(None)
        dyn_titles.append(None)
        dyn_abstracts.append(None)   
    
    # if we are at critical number of successes, save files and start new files over 
    if successes % success_cutoff == 0 and attempts > 0:
        print('wrote file at ',successes,' successes')
        meta_df = pd.DataFrame(dyn_urls,columns = ['url'])
        meta_df['pii'] = dyn_piis
        meta_df['year'] = dyn_years
        meta_df['success'] = dyn_successes
        meta_df['issn'] = dyn_issns
        meta_df['journal'] = dyn_journals
        meta_df['title'] = dyn_titles
        meta_df['abstract'] = dyn_abstracts
        meta_df['authors'] = dyn_auths
        
        meta_filename = 'corpus_metadata_' + str(filenum) + '.json'
        meta_df.to_json(meta_filename)
    
        file.close()                                            # close the current file so it saves 
        text_filename = 'corpus_texts_' + str(filenum + 1) + '.txt' # new text file number is +1 ahead of the metadata file number because a file was created before the loop, can probably fix this later
        file = open(text_filename,'w')                              # the new file being written to
        
        filenum += 1 

Attempts :  1
Successes:  0
resetting the dynamic lists
Attempts :  2
Successes:  1
Attempts :  3
Successes:  2
Attempts :  4
Successes:  3
Attempts :  5
Successes:  4
Attempts :  6
Successes:  5
Attempts :  7
Successes:  6
Attempts :  8
Successes:  7
Attempts :  9
Successes:  8
Attempts :  10
Successes:  9
Attempts :  11
Successes:  10
Attempts :  12
Successes:  11
Attempts :  13
Successes:  12
Attempts :  14
Successes:  13
Attempts :  15
Successes:  14
Attempts :  16
Successes:  15
Attempts :  17
Successes:  16
Attempts :  18
Successes:  17
Attempts :  19
Successes:  18
Attempts :  20
Successes:  19
Attempts :  21
Successes:  20
Attempts :  22
Successes:  21
Attempts :  23
Successes:  22
Attempts :  24
Successes:  23
Attempts :  25
Successes:  24
Attempts :  26
Successes:  25
Attempts :  27
Successes:  26
Attempts :  28
Successes:  27
Attempts :  29
Successes:  28
Attempts :  30
Successes:  29
Attempts :  31
Successes:  30
Attempts :  32
Successes:  31
Attempts :  33
Successes:  32
A

KeyError: 73129

In [20]:
df['url'][73131]

'https://www-sciencedirect-com.offcampus.lib.washington.edu/science/article/pii/S0009261499000329'

## Below here is spot checking the output to see if the code is working 

In [1]:
import numpy as np
import pandas as pd
import os
output_filepath = '/Volumes/My Passport/Davids Stuff/'
os.chdir(output_filepath)

In [2]:
spot_df = pd.read_json('corpus_metadata_4.json')
spot_file = open('corpus_texts_4.txt','r')
spot_texts = []
for text in spot_file:
    spot_texts.append(text)

In [3]:
spot_df.head()

,url,pii,year,success,issn,journal,title,abstract,authors
0,https://www-sciencedirect-com.offcampus.lib.wa...,S2214993717300969,2018,1,22149937,Sustainable Materials and Technologies,Critical raw materials: A perspective from the...,Functional materials are crucial to meet today...,"[Hofmann, Margarethe, Hofmann, Heinrich, Hagel..."
1,https://www-sciencedirect-com.offcampus.lib.wa...,S2214993718300964,2018,1,22149937,Sustainable Materials and Technologies,Self-assembly of yolk-shell porous Fe-doped g-...,A facile and environmentally benign microwave ...,"[Yu, Yongzhi, Cheng, Si, Wang, Leying, Zhu, We..."
2,https://www-sciencedirect-com.offcampus.lib.wa...,S2214993718301908,2018,1,22149937,Sustainable Materials and Technologies,Temperature-dependent evolution of hydroxyl ra...,Rational regulation of redox capacity in advan...,"[Duan, Xiaoguang, Indrawirawan, Stacey, Kang, ..."
3,https://www-sciencedirect-com.offcampus.lib.wa...,S2214993718300927,2018,1,22149937,Sustainable Materials and Technologies,Novel ecosustainable peat and oil shale ash-ba...,"In this study, a novel peat composite construc...","[Liiv, Jüri, Teppand, Tõnis, Rikmann, Ergo, Te..."
4,https://www-sciencedirect-com.offcampus.lib.wa...,S2214993718301271,2018,1,22149937,Sustainable Materials and Technologies,A sustainable pavement concrete using warm mix...,"Recently, increasing material prices coupled w...","[Albayati, Amjad, Wang, Yu, Wang, Yan, Haynes,..."


In [4]:
spot_df.iloc[0,0]

'https://www-sciencedirect-com.offcampus.lib.washington.edu/science/article/pii/S2214993717300969'

In [5]:
spot_df.iloc[0,8][0]

'Hofmann, Margarethe'

In [28]:
spot_texts[0]

"serial JL 306205 291210 291719 291731 291777 291884 31 Sustainable Materials and Technologies SUSTAINABLEMATERIALSTECHNOLOGIES 2018-08-10 2018-08-10 2018-09-04 2018-09-04 2018-09-04T03:56:11 1-s2.0-S2214993717300969 S2214-9937(17)30096-9 S2214993717300969 10.1016/j.susmat.2018.e00074 S300 S300.1 FULL-TEXT 1-s2.0-S2214993718X00049 2018-09-04T03:23:43.812246Z 0 0 20180901 20180930 2018 2018-08-10T15:28:27.822477Z articleinfo articlenumber articletitlenorm authfirstinitialnorm authfirstsurnamenorm cid cids contenttype copyright crossmark dateloaded dateloadedtxt datesearch datesort dateupdated dco docsubtype doctype doi eid ewtransactionid hubeid indexeddate issn issnnorm itemstage itemtransactionid itemweight openaccess openarchive pg pgfirst pii piinorm pubdateend pubdatestart pubdatetxt pubyr sortorder srctitle srctitlenorm srctype ssids alllist content subj subheadings suppl tomb volfirst volissue volumelist webpdf webpdfpagecount yearnav figure table body acknowledge affil articleti